In [ ]:
import requests
import json

# Uber API Credentials (Sandbox Mode)
UBER_CLIENT_SECRET= "123"
UBER_CLIENT_ID= "213"

# Uber API Endpoints
UBER_TOKEN_URL = "https://auth.uber.com/oauth/v2/token"
UBER_SANDBOX_URL = "https://sandbox-api.uber.com/v1.2"
UBER_PRICE_ESTIMATE_URL = f"{UBER_SANDBOX_URL}/estimates/price"
UBER_REQUEST_URL = f"{UBER_SANDBOX_URL}/requests"

# Test Coordinates (Start and End Locations)
START_LAT, START_LON = 38.65150802238675, -90.31460075914723
END_LAT, END_LON = 38.64041365328198, -90.30299215925946

# Step 1: Get OAuth Access Token
def get_uber_access_token():
    data = {
        "client_id": UBER_CLIENT_ID,
        "client_secret": UBER_CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "request"
    }
    response = requests.post(UBER_TOKEN_URL, data=data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token: {response.json()}")



In [11]:
print("Fetching Uber Access Token...")
token = get_uber_access_token()
print("Access Token Retrieved ✅")

Fetching Uber Access Token...


Exception: Failed to get access token: {'error': 'invalid_scope', 'error_description': 'scope(s) are invalid'}

In [ ]:
# Step 2: Get Price Estimates (Sandbox)
def get_price_estimate(access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    params = {
        "start_latitude": START_LAT,
        "start_longitude": START_LON,
        "end_latitude": END_LAT,
        "end_longitude": END_LON,
        "seat_count": 1
    }

    response = requests.get(UBER_PRICE_ESTIMATE_URL, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Price estimate failed: {response.json()}")

# Step 3: Request a Ride (Sandbox)
def request_ride(access_token, product_id):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "product_id": product_id,
        "start_latitude": START_LAT,
        "start_longitude": START_LON,
        "end_latitude": END_LAT,
        "end_longitude": END_LON,
        "seat_count": 1
    }

    response = requests.post(UBER_REQUEST_URL, headers=headers, json=data)

    if response.status_code == 202:
        return response.json()
    else:
        raise Exception(f"Ride request failed: {response.json()}")

# Step 4: Get Ride Details (Sandbox)
def get_ride_details(access_token, request_id):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(f"{UBER_REQUEST_URL}/{request_id}", headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to get ride details: {response.json()}")

# Step 5: Test Sandbox Workflow
if __name__ == "__main__":
    try:
        print("Fetching Uber Access Token...")
        token = get_uber_access_token()
        print("Access Token Retrieved ✅")

        print("\nGetting price estimates...")
        price_data = get_price_estimate(token)
        print(json.dumps(price_data, indent=2))

        # Extract a product_id for ride request
        product_id = price_data["prices"][0]["product_id"]

        print("\nRequesting a sandbox ride...")
        ride_request = request_ride(token, product_id)
        request_id = ride_request["request_id"]
        print("Ride requested successfully ✅")

        print("\nFetching ride details...")
        ride_details = get_ride_details(token, request_id)
        print(json.dumps(ride_details, indent=2))

    except Exception as e:
        print(f"Error: {e}")